github link: [czip](https://github.com/DingWB/czip)

In [2]:
import os,sys
%alias czip /anvil/projects/x-mcb130189/Wubin/Software/miniconda3/envs/yap/bin/czip

## Generate 2D subset index for a genomic region
Instead of generating 1D subset index for CGN, CHN, in which, the IDs for the specific pattern will be written into the resulting .ssi file.
2D subset index will create a .ssi file including a genomic region, ID start and ID end will be written.

In [3]:
czip generate_ssi2 --help

INFO: Showing help with the command 'czip generate_ssi2 -- --help'.

NAME
    czip generate_ssi2 - Generate subset index for a genomic region bed file. For example: czip generate_ssi2 -i ~/Ref/mm10/annotations/mm10_with_chrL.allc.cz         -o mm10_with_chrL.allc.genes_flank2k.ssi -b genes_flank2k.bed.gz -n 4

SYNOPSIS
    czip generate_ssi2 INPUT <flags>

DESCRIPTION
    Generate subset index for a genomic region bed file. For example: czip generate_ssi2 -i ~/Ref/mm10/annotations/mm10_with_chrL.allc.cz         -o mm10_with_chrL.allc.genes_flank2k.ssi -b genes_flank2k.bed.gz -n 4

POSITIONAL ARGUMENTS
    INPUT

FLAGS
    -o, --output=OUTPUT
        Type: Optional[]
        Default: None
    -b, --bed=BED
        Type: Optional[]
        Default: None
    -n, --n_jobs=N_JOBS
        Default: 4

NOTES
    You can also use flags syntax for POSITIONAL ARGUMENTS


```shell
czip generate_ssi2 -i ~/Ref/mm10/annotations/mm10_with_chrL.allc.cz \
        -o mm10_with_chrL.allc.genes_flank2k.ssi -b genes_flank2k.bed.gz -n 4
```

## Aggregate genomic regions

In [4]:
czip aggregate --help

INFO: Showing help with the command 'czip aggregate -- --help'.

NAME
    czip aggregate - Aggregate a given genomic region on a .cz file, for example: /usr/bin/time -f "%e    %M      %P" czip aggregate -I test.cz -O test_gene.cz         -s mm10_with_chrL.allc.genes_flank2k.ssi

SYNOPSIS
    czip aggregate <flags>

DESCRIPTION
    Aggregate a given genomic region on a .cz file, for example: /usr/bin/time -f "%e    %M      %P" czip aggregate -I test.cz -O test_gene.cz         -s mm10_with_chrL.allc.genes_flank2k.ssi

FLAGS
    -I, --Input=INPUT
        Type: Optional[]
        Default: None
    -O, --Outfile=OUTFILE
        Type: Optional[]
        Default: None
    -s, --ssi=SSI
        Type: Optional[]
        Default: None
    -i, --intersect=INTERSECT
        Type: Optional[]
        Default: None
    -e, --exclude=EXCLUDE
        Type: Optional[]
        Default: None
    -c, --chunksize=CHUNKSIZE
        Default: 5000
    -f, --formats=FORMATS
        Default: ['H', 'H']


```shell
czip aggregate -I test.cz -O test_gene.cz \
        -s mm10_with_chrL.allc.genes_flank2k.ssi
```

In the resulting test_gene.cz, each row is a region corresponding to one gene, mc and cov are summed up for all CG/CH located at this gene.